# Pruebas de la búsqueda de secuencias con todas las posiciones y ángulos de los esqueletos

En este cuaderno se van a mostrar una serie de pruebas de como el algoritmo encuentra la secuencia correspondiente a un ejercicio dentro de una secuencia de mayor tamaño obtenida a partir de un video que realiza varios ejercicios seguidos.

Esto se realizará una vez se hayan obtenido los ángulos de todos los frames, tanto de los ejercicios individuales como del vídeo que realiza éstos ejercicios de manera ininterrumpida. Para ello se han planteado varias soluciones:
- Solución 1: Se trabajará con arrays unidimensionales,
- Solución 2: Se buscarán secuencias de inicio y de final,
- Solución 3: "

### Planteamiento del problema

Con las posiciones localizadas en las que comienza y finalizan los ejercicios dentro de la secuencia larga se procederá de la siguiente manera. Sabemos que el vídeo en el que se realizan todos los ejercicios tiene \"x\" frames y sabemos en que segundo del vídeo se empieza y acaba un ejercicio (esto lo observamos al reproducir el vídeo, no es algo que haya obtenido del programa). Con esta información localizaremos en que puntos empiezan y acaban cada uno de los ejercicios y los compararemos con los resultados obtenidos en el programa

Según se ha observado en el vídeo los ejercicios comienzan en los siguientes instantes de tiempo:
- Ejercicio 1: 0 - 0:32 
- Ejercicio 2: 0:32 - 0:51 
- Ejercicio 3: 0:51 - 1:12
- Ejercicio 4: 1:12 - 1:37

Por lo tanto lo que se tendrá que hacer es dividir el número de frames obtenidos entre el tiempo del vídeo y de esta manera se podrá hacer una aproximación de los frames que ha obtenido en cada segundo.
- El número de frames obtenidos en el vídeo largo han sido <b> 2919 frames </b>
- Como la duración del vídeo es de 1:37 min tenemos una duración de <b> 97 segundos </b>
- Por lo tanto <b> en cada segundo se han obtenido 30 frames </b>, esto quiere decir:
    - Ejericio 1 comprede desde el frame 0 hasta el frame 960
    - Ejericio 2 comprede desde el frame 960 hasta el frame 1530
    - Ejericio 3 comprede desde el frame 1530 hasta el frame 2160
    - Ejericio 4 comprede desde el frame 2160 hasta el frame 2919"

In [3]:
# imports
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import math
import os
import pandas as pd
import pickle as pk
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

# import de utilities de detectron
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

#modelo
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.99  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

from PosicionVF import Posicion #Esta clase nos permitirá sacar los ángulos de cada esqueleto

Para mostrar los ejemplos, se va a empezar cargando todos los valores de las posiciones de los ejercicios individuales y de los del video que contiene todos los movimientos.

Los ángulos se han tenido que almacenar de la siguiente forma: [ángulo ,0]. Esto se debe a que si no por cada esqueleto generaría una matriz con valores multidimensionales y unidimensionales, lo que más tarde implicaría un problema en el cálculo de las distancias

In [4]:
def obtenerPosiciones(directorio):
    contenido=os.listdir(directorio)
    
    array_puntos=[]

    for imagen in contenido:
        imagen2=directorio+imagen
        imagen3=cv2.imread(imagen2)

        out = predictor(imagen3)
        type(out)
        j = out.get("instances")
        pk = j.pred_keypoints.cpu()

        if len(pk)>0:
            x = pk[0][:,0].cpu().numpy()
            y = pk[0][:,1].cpu().numpy()

            obj1=Posicion(x,y)
            array_puntos.append([obj1.nariz, obj1.hombroI, obj1.hombroD, obj1.cuello, [obj1.angCuelloSupI,0] ,
                                [obj1.angCuelloSupD,0], obj1.codoI, obj1.codoD, obj1.manoI, obj1.manoD,
                                [obj1.angCodoI,0], [obj1.angCodoD,0], [obj1.angHombroI,0], [obj1.angHombroD,0],
                                 obj1.caderaI, obj1.caderaD, obj1.cadera, obj1.rodillaI, obj1.rodillaD ,
                                [obj1.angCaderaI,0], [obj1.angCaderaD,0], [obj1.angCaderaTorsoI,0],
                                [obj1.angCaderaTorsoD,0], obj1.tobilloI, obj1.tobilloD ,[obj1.angRodillaI,0] ,
                                [obj1.angRodillaD,0] ])

    return array_puntos

In [7]:
directorio='/home/lnc1002/frames_ejercicio1/' 
array_puntos1=obtenerPosiciones(directorio)
directorio='/home/lnc1002/frames_ejercicio2/' 
array_puntos2=obtenerPosiciones(directorio)
directorio='/home/lnc1002/frames_ejercicio3/' 
array_puntos3=obtenerPosiciones(directorio)
directorio='/home/lnc1002/frames_ejercicio4/' 
array_puntos4=obtenerPosiciones(directorio)
directorio='/home/lnc1002/frames_ejercicios_totales/' 
array_puntos_totales=obtenerPosiciones(directorio)

## Solución 1: Trabajar con arrays unidimensionales

Lo siguiente que vamos a hacer es convertir los arrays multidimensionales en arrays unidimensionales.

In [9]:
array_puntos1_3=np.array(array_puntos1).flatten()
array_puntos2_3=np.array(array_puntos2).flatten()
array_puntos3_3=np.array(array_puntos3).flatten()
array_puntos4_3=np.array(array_puntos4).flatten()
array_puntos_total_3=np.array(array_puntos_totales).flatten()

print("Ejercicio 1: ",len(array_puntos1_3))
print("Ejercicio 2: ",len(array_puntos2_3))
print("Ejercicio 3: ",len(array_puntos3_3))
print("Ejercicio 4: ",len(array_puntos4_3))
print("Ejercicio completo: ",len(array_puntos_total_3))

print("Longitud de los ejercicios por separado: ",len(array_puntos1_3)+len(array_puntos2_3)+len(array_puntos3_3)+len(array_puntos4_3))

Ejercicio 1:  52326
Ejercicio 2:  33696
Ejercicio 3:  33804
Ejercicio 4:  23706
Ejercicio completo:  156438
Longitud de los ejercicios por separado:  143532


<div class="alert alert-block alert-warning">
    <b>Nota: </b>Como se puede observar la longitud del vídeo que contiene todos los ejercicios no es igual a la suma de los diferentes ejercicios, es por esa razón que puede haber frames sin adjudicar a ningún ejercicio
</div>

Una vez que tenemos todos los valores, un aspecto MUY IMPORTANTE es eliminar aquellos valores 'nan'. En caso de no ser eliminados, el algoritmo interpreta que este es el camino más óptimo y detecta ese frame como inicio y final del ejercicio de referencia

In [10]:
array_puntos_total_3 = array_puntos_total_3[~np.isnan(array_puntos_total_3)]

Una vez eliminados los valores habrá que tener en cuenta lo siguiene. En el vídeo de ejercicios completos no aparecen sergún el orden de ejercicios individuales, es decir, los ejercicios se han realizado de la siguiente manera:
- Primer ejercicio del vídeo completo  -> Ejercicio 1
- Segundo ejercicio del vídeo completo -> Ejercicio 4
- Tercer ejercicio del vídeo completo  -> Ejercicio 2
- Cuarto ejercicio del vídeo completo  -> Ejercicio 3

Por lo que, teniendo en cuenta los valores almacenados de cada ejercicio:
- Ejercicio 1: 0-52.326
- Ejercicio 2: 76.032-109.728
- Ejercicio 3: 109.728-143.532
- Ejercicio 4: 52.326-76.032   

Estos valores son bastante relativos porque ya hemos comentado que entre vídeos los ejercicios no duran lo mismo ni empieza uno nada más terminar el siguiente.

In [11]:
from tslearn.metrics import dtw_subsequence_path

path, dist = dtw_subsequence_path(array_puntos1_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 1 comienza en el frame =",a_ast)
print("El Ejercicio 1 finaliza en el frame =",b_ast)

El Ejercicio 1 comienza en el frame = 162
El Ejercicio 1 finaliza en el frame = 52487


<div class="alert alert-block alert-success">
    Es increible, encuentra la secuencia prácticamente a la perfección.
</div>

In [12]:
path, dist = dtw_subsequence_path(array_puntos2_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 2 comienza en el frame =",a_ast)
print("El Ejercicio 2 finaliza en el frame =",b_ast)

El Ejercicio 2 comienza en el frame = 18522
El Ejercicio 2 finaliza en el frame = 52217


<div class="alert alert-block alert-danger">
    El resultado obtenido está demasiado lejos del valor esperado.
</div>

In [13]:
path, dist = dtw_subsequence_path(array_puntos3_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 3 comienza en el frame =",a_ast)
print("El Ejercicio 3 finaliza en el frame =",b_ast)

El Ejercicio 3 comienza en el frame = 49626
El Ejercicio 3 finaliza en el frame = 83265


<div class="alert alert-block alert-danger">
    El resultado obtenido está demasiado lejos del valor esperado.
</div>

In [15]:
path, dist = dtw_subsequence_path(array_puntos4_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 4 comienza en el frame =",a_ast)
print("El Ejercicio 4 finaliza en el frame =",b_ast)

El Ejercicio 4 comienza en el frame = 14148
El Ejercicio 4 finaliza en el frame = 37853


<div class="alert alert-block alert-danger">
    El resultado obtenido está demasiado lejos del valor esperado.
</div>

<div class="alert alert-block alert-danger">
    Como hemos podido observar sólo encuentra bien la primera secuencia pero el resto está bastante lejos de encontrarlas. No podemos estimar el resto a dependencias de la primera porque por los datos observados no se sabe si la ha encontrado por su buena semejanza o simplemente por pura coincidencia.
</div>

### Recortamos los Frames

In [19]:
def cutFrames(array_puntos_largo):
    array_puntos=[]
    guardar=0
    for ang in array_puntos_largo:
        guardar=guardar+1
        if guardar==3:
            array_puntos.append(ang)
            guardar=0
    return array_puntos

array_puntos1_2=cutFrames(array_puntos1)
print("Nuevo número total de frames para el Ejercicio 1: ", len(array_puntos1_2))    

array_puntos2_2=cutFrames(array_puntos2)
print("Nuevo número total de frames para el Ejercicio 2: ", len(array_puntos2_2)) 

array_puntos3_2=cutFrames(array_puntos3)
print("Nuevo número total de frames para el Ejercicio 3: ", len(array_puntos3_2)) 

array_puntos4_2=cutFrames(array_puntos4)
print("Nuevo número total de frames para el Ejercicio 4: ", len(array_puntos4_2)) 

array_puntos_total_2=cutFrames(array_puntos_totales) 
print("Nuevo número total de frames para los Ejercicios totales: ", len(array_puntos_total_2)) 

Nuevo número total de frames para el Ejercicio 1:  323
Nuevo número total de frames para el Ejercicio 2:  208
Nuevo número total de frames para el Ejercicio 3:  208
Nuevo número total de frames para el Ejercicio 4:  146
Nuevo número total de frames para los Ejercicios totales:  965


Con estos valores repetimos el experimento anterior

In [20]:
array_puntos1_3=np.array(array_puntos1_2).flatten()
array_puntos2_3=np.array(array_puntos2_2).flatten()
array_puntos3_3=np.array(array_puntos3_2).flatten()
array_puntos4_3=np.array(array_puntos4_2).flatten()
array_puntos_total_3=np.array(array_puntos_total_2).flatten()

print("Ejercicio 1: ",len(array_puntos1_3))
print("Ejercicio 2: ",len(array_puntos2_3))
print("Ejercicio 3: ",len(array_puntos3_3))
print("Ejercicio 4: ",len(array_puntos4_3))
print("Ejercicio completo: ",len(array_puntos_total_3))

print("Longitud de los ejercicios por separado: ",len(array_puntos1_3)+len(array_puntos2_3)+len(array_puntos3_3)+len(array_puntos4_3))

array_puntos_total_3 = array_puntos_total_3[~np.isnan(array_puntos_total_3)]

Ejercicio 1:  17442
Ejercicio 2:  11232
Ejercicio 3:  11232
Ejercicio 4:  7884
Ejercicio completo:  52110
Longitud de los ejercicios por separado:  47790


Teniendo en cuenta los valores almacenados de cada ejercicio:
- Ejercicio 1: 0-17.442
- Ejercicio 2: 25.326 -36.558
- Ejercicio 3: 36.558-47.790
- Ejercicio 4: 17.442-25.326 

In [21]:
path, dist = dtw_subsequence_path(array_puntos1_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 1 comienza en el frame =",a_ast)
print("El Ejercicio 1 finaliza en el frame =",b_ast)

El Ejercicio 1 comienza en el frame = 12366
El Ejercicio 1 finaliza en el frame = 29807


<div class="alert alert-block alert-danger">
    El resultado obtenido está demasiado lejos del valor esperado.
</div>

In [22]:
path, dist = dtw_subsequence_path(array_puntos2_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 2 comienza en el frame =",a_ast)
print("El Ejercicio 2 finaliza en el frame =",b_ast)

El Ejercicio 2 comienza en el frame = 16632
El Ejercicio 2 finaliza en el frame = 27863


<div class="alert alert-block alert-info">
    El resultado obtenido está lejos de ser el esperado pero podría llegar a ser aceptable
</div>

In [23]:
path, dist = dtw_subsequence_path(array_puntos3_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 3 comienza en el frame =",a_ast)
print("El Ejercicio 3 finaliza en el frame =",b_ast)

El Ejercicio 3 comienza en el frame = 35322
El Ejercicio 3 finaliza en el frame = 46439


<div class="alert alert-block alert-success">
    Es increible, encuentra la secuencia prácticamente a la perfección.
</div>

In [24]:
path, dist = dtw_subsequence_path(array_puntos4_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 4 comienza en el frame =",a_ast)
print("El Ejercicio 4 finaliza en el frame =",b_ast)

El Ejercicio 4 comienza en el frame = 15174
El Ejercicio 4 finaliza en el frame = 23057


<div class="alert alert-block alert-info">
    El resultado obtenido está lejos de ser el esperado pero podría llegar a ser aceptable
</div>

### Recortamos aún más los Frames 

In [25]:
def cutFrames(array_puntos_largo):
    array_puntos=[]
    guardar=0
    for ang in array_puntos_largo:
        guardar=guardar+1
        if guardar==6:
            array_puntos.append(ang)
            guardar=0
    return array_puntos

array_puntos1_2=cutFrames(array_puntos1)
print("Nuevo número total de frames para el Ejercicio 1: ", len(array_puntos1_2))    

array_puntos2_2=cutFrames(array_puntos2)
print("Nuevo número total de frames para el Ejercicio 2: ", len(array_puntos2_2)) 

array_puntos3_2=cutFrames(array_puntos3)
print("Nuevo número total de frames para el Ejercicio 3: ", len(array_puntos3_2)) 

array_puntos4_2=cutFrames(array_puntos4)
print("Nuevo número total de frames para el Ejercicio 4: ", len(array_puntos4_2)) 

array_puntos_total_2=cutFrames(array_puntos_totales) 
print("Nuevo número total de frames para los Ejercicios totales: ", len(array_puntos_total_2))

Nuevo número total de frames para el Ejercicio 1:  161
Nuevo número total de frames para el Ejercicio 2:  104
Nuevo número total de frames para el Ejercicio 3:  104
Nuevo número total de frames para el Ejercicio 4:  73
Nuevo número total de frames para los Ejercicios totales:  482


Con estos valores repetimos el experimento anterior

In [26]:
array_puntos1_3=np.array(array_puntos1_2).flatten()
array_puntos2_3=np.array(array_puntos2_2).flatten()
array_puntos3_3=np.array(array_puntos3_2).flatten()
array_puntos4_3=np.array(array_puntos4_2).flatten()
array_puntos_total_3=np.array(array_puntos_total_2).flatten()

print("Ejercicio 1: ",len(array_puntos1_3))
print("Ejercicio 2: ",len(array_puntos2_3))
print("Ejercicio 3: ",len(array_puntos3_3))
print("Ejercicio 4: ",len(array_puntos4_3))
print("Ejercicio completo: ",len(array_puntos_total_3))

print("Longitud de los ejercicios por separado: ",len(array_puntos1_3)+len(array_puntos2_3)+len(array_puntos3_3)+len(array_puntos4_3))

array_puntos_total_3 = array_puntos_total_3[~np.isnan(array_puntos_total_3)]

Ejercicio 1:  8694
Ejercicio 2:  5616
Ejercicio 3:  5616
Ejercicio 4:  3942
Ejercicio completo:  26028
Longitud de los ejercicios por separado:  23868


Teniendo en cuenta los valores almacenados de cada ejercicio:
- Ejercicio 1: 0-8.694
- Ejercicio 2: 12.636-18.252
- Ejercicio 3: 18.252-23.868
- Ejercicio 4: 8.694-12.636 

In [31]:
path, dist = dtw_subsequence_path(array_puntos1_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 1 comienza en el frame =",a_ast)
print("El Ejercicio 1 finaliza en el frame =",b_ast)

El Ejercicio 1 comienza en el frame = 5076
El Ejercicio 1 finaliza en el frame = 13769


<div class="alert alert-block alert-info">
    El resultado obtenido está lejos de ser el esperado pero podría llegar a ser aceptable
</div>

In [32]:
path, dist = dtw_subsequence_path(array_puntos2_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 2 comienza en el frame =",a_ast)
print("El Ejercicio 2 finaliza en el frame =",b_ast)

El Ejercicio 2 comienza en el frame = 8316
El Ejercicio 2 finaliza en el frame = 13931


<div class="alert alert-block alert-info">
    El resultado obtenido está ALGO lejos del valor esperado.
</div>

In [33]:
path, dist = dtw_subsequence_path(array_puntos3_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 3 comienza en el frame =",a_ast)
print("El Ejercicio 3 finaliza en el frame =",b_ast)

El Ejercicio 3 comienza en el frame = 16686
El Ejercicio 3 finaliza en el frame = 22301


<div class="alert alert-block alert-success">
    Es increible, encuentra la secuencia prácticamente a la perfección.
</div>

In [34]:
path, dist = dtw_subsequence_path(array_puntos4_3, array_puntos_total_3)

path=np.array(path)
a_ast = path[0, 1]
b_ast = path[-1, 1]

print("El Ejercicio 4 comienza en el frame =",a_ast)
print("El Ejercicio 4 finaliza en el frame =",b_ast)

El Ejercicio 4 comienza en el frame = 9936
El Ejercicio 4 finaliza en el frame = 13877


<div class="alert alert-block alert-success">
    Es increible, encuentra la secuencia prácticamente a la perfección.
</div>

<div class="alert alert-block alert-warning">
    <b>Conclusión: </b>Como se puede observar a menor número de frames, mejor encuentra algunas secuencias.
    Hay que añadir que además el tiempo de ejecución es mucho mejor pero como se puede observar en los ejemplos, encuentra algunas bien y otras demasiado mal por lo que no es una herramienta del todo fiable
</div>